# Name that Speaker
> Using fastai to guess who might say a particular phrase in a chat conversation

- toc: true
- badges: true
- comments: true
- author: Matt Bowen
- categories: [jupyter]
- comments: true

## Intro

The first few lessons of the [fastai course](https://course.fast.ai/) lean heavily towards computer vision problems with their examples. Personally, I am a little more interested in natural language processing and work with text applications, so I glommed onto their example of doing sentiment analysis of movie reviews using fastai. 

Here's how they built that model using the IMDB dataset internal to the library:

In [1]:
from fastai.text.all import *

dls = TextDataLoaders.from_folder(untar_data(URLs.IMDB), valid='test', encoding='utf8', bs=32)
learn = text_classifier_learner(dls, AWD_LSTM, drop_mult=0.5, metrics=accuracy)
learn.fine_tune(4, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.611722,0.397967,0.820560,07:53


epoch,train_loss,valid_loss,accuracy,time
0,0.304293,0.294640,0.875800,16:06
1,0.280457,0.206577,0.920880,16:07
2,0.201380,0.181090,0.929840,16:08
3,0.153116,0.179792,0.931280,16:05


The generated model `learn` can then be used to predict the sentiment of a statement. I picked three statements below to show what it's predictions are like. The model predicts the first two statements accurately and is fairly confident in its prediction. For the third, the model predicts the sentiment, but isn't as confident, which is not surprising since I picked that one to be intentionally tricky.

In [4]:
x = learn.predict("I really liked that movie!")
y = learn.predict("At no point in your rambling, incoherent response was there anything that could even be considered a rational thought. Everyone in this room is now dumber for having listened to it. I award you no points, and may God have mercy on your soul.")
z = learn.predict("I thought it was going to be good, but it really was not in the end.")
print(f"Sentiment of x: {x[0]}, prob={x[2][1]:.4f}")
print(f"Sentiment of y: {y[0]}, prob={y[2][0]:.4f}")
print(f"Sentiment of z: {z[0]}, prob={z[2][0]:.4f}")

Sentiment of x: pos, prob=0.9987
Sentiment of y: neg, prob=0.9659
Sentiment of z: neg, prob=0.7692


My idea was to take this template for building a text classification model and use it to classify the "speaker" of a given statement, given a previous set of chat conversations to train on.

## Data Preparation

In a previous post, I took some data from a Google Hangouts chat and converted it to a format more palatable to feeding into a PyTorch LSTM, i.e. each chat message was broken up to be in the format 
> Speaker :: Message

I'm going to use the same underlying data here, but format it slightly differently to ease import into [fastai](https://www.fast.ai/). This might not be the cleanest way to do this, but it worked :smile:

The format I ended up using was a modified csv. Commas are pretty prevalent in the data and I hate using quotes and escapes, so I used `|` to separate the columns {% fn 1 %}. Since I had already done the separation of speaker and message using `::` before, the script to convert was fairly straightforward, minus one spot where someone had used an SAT-style analogy 
> Kappa :: Omega:OK :: Gamma:"Here's the thing"

{{ 'Which appeared in the corpus as || (aka OR)' | fndetail: 1 }}


In [17]:
chat_filename = "/notebooks/fastbook/chat/chatFile.txt"
chat_csv = "/notebooks/fastbook/chat/chatFile.csv"

# Read in the chat file with 
data = open(chat_filename, encoding='utf8').read()
# As software developers, we used "||" a few places to mean OR
data = data.replace("||", "or")
data = data.splitlines()

# Write to csv
with open(chat_csv, encoding='utf8', mode='w') as csv:
    # Header
    csv.write("Name|Message")
    # New message
    for line in data:
        if "::" in line:
            x = line.split("::")
            if len(x) > 2:
                (name, msg) = ("Kappa", "Omega:Ok :: Gamma:Here's the thing")
            else:
                (name, msg) = line.split("::")
            name = name.strip()
            msg = msg.strip()
            csv.write('\n')
            csv.write(name)
            csv.write("|")
            csv.write(msg)
        else:
            csv.write(" " + msg)
    csv.write('\n')

## Build Model

The csv now matched each message to a particular speaker in a format that was easily digestible by `fastai`. Next, I mimicked the sentimental analysis example above to make my speaker identification model. I'm essentially just swapping `from_folder` out for `from_csv`, with some extra arguments to give details about my csv.

In [18]:
from fastai.text.all import *

dls = TextDataLoaders.from_csv('.', csv_fname=chat_csv, 
                               delimiter="|", text_col = 1, label_col = 0)
learn = text_classifier_learner(dls, AWD_LSTM, drop_mult=0.5, metrics=accuracy)
learn.fine_tune(4, 1e-2)

/opt/conda/envs/fastai/lib/python3.8/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


epoch,train_loss,valid_loss,accuracy,time
0,1.567062,1.340145,0.449983,00:16


epoch,train_loss,valid_loss,accuracy,time
0,1.305043,1.235283,0.500000,00:38
1,1.228520,1.160044,0.540014,00:37
2,1.107468,1.124508,0.564677,00:37
3,1.059996,1.121178,0.570369,00:37


Save the model to a file for later use

In [19]:
learn.export("/notebooks/fastbook/chat/chat_model.pkl")

## My First App

The challenge in the second lesson of the `fastai` course was to create a model using `fastai` and turn it into a prototype web app. The structure of how to do so using `ipywidgets` and `voila` was pretty straightforward.

A box for giving the text to evaluate

In [20]:
import ipywidgets as widgets
txtInput = widgets.Textarea(placeholder='Input text...', description='Text:')
txtInput

Textarea(value='', description='Text:', placeholder='Input text...')

A button to execute the prediction for the model

In [21]:
button = widgets.Button(description='Predict',
                        tooltip='Click me',
                        icon='question')
button

Button(description='Predict', icon='question', style=ButtonStyle(), tooltip='Click me')

Set up the output widget with a dividing line

In [22]:
outWidget = widgets.Output(layout={'border': '1px solid black'})
outWidget

Output(layout=Layout(border='1px solid black'))

In [23]:
def on_click_classify(change):
    # predictions and probabilities from the model
    prediction, idx, probs = learn_inf.predict(txtInput.value)   
    # pair the probabilities with each speaker
    outputs = list(zip(probs, learn_inf.dls.vocab[1]))
    # sort the list with the most likely speaker first
    outputs.sort(reverse=True)
    outWidget.clear_output()
    # Print the output, with the most likely speaker in bold
    with outWidget:
        header = widgets.HTML()
        header.value = '<u>Scores</u>'
        display(header)
        lblPred = widgets.HTML()
        lblPred.value = f'<b>{outputs[0][1]}</b>: <b>{100 * outputs[0][0]:.2f}%</b>'
        display(lblPred)
        for (prob, name) in outputs[1:]:
            lbl = widgets.Label()
            lbl.value = f'{name}:  {100 * prob:.2f}%'
            display(lbl)

button.on_click(on_click_classify)

### Shortcoming

One obvious shortcoming of this speaker identification model is that one of the speakers ('Kappa') was much more likely to be identified as the most likely speaker than any of the other speakers for almost any text. He accounts for about 44% of the input messages, but I wasn't sure how (or even if I should) adjust for that.

### Failure to Launch

I was able to run Voila locally in my notebook and get it to produce a viable web app. Unfortunately, I was unable to get it to host properly on [Heroku](https://www.heroku.com/), as suggested in the course. All I could seem to get was a nebulous "Application Error" and did not have the time or patience to wade through figuring it out.

I have some evidence to think that the issue was the OS differences between the Paperspace notebooks that I was using for fastai development, the Windows environment I hosted the Jupyter notebook (and ultimately got the app running locally), and whatever Heroku is running on their server. These differences preventing a model built in one place from working in another and couldn't actually build the model on Heroku.